Group Project

In [24]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   --- ------------------------------------ 1.6/16.1 MB 9.3 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/16.1 MB 9.9 MB/s eta 0:00:02
   ------------- -------------------------- 5.2/16.1 MB 8.2 MB/s eta 0:00:02
   ------------------ --------------------- 7.6/16.1 MB 9.4 MB/s eta 0:00:01
   ------------------------ --------------- 9.7/16.1 MB 9.4 MB/s eta 0:00:01
   ------------------------------ --------- 12.3/16.1 MB 9.9 MB/s eta 0:00:01
   ------------------------------------- -- 14.9/16.1 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 10.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [39]:
import os
from dotenv import load_dotenv
import mysql.connector
import requests
import pandas as pd
from datetime import datetime, timedelta


In [14]:
load_dotenv()  # Load environment variables from .env file
TICKETMASTER_API_KEY = os.getenv('TICKETMASTER_API_KEY')
TOMORROW_API_KEY = os.getenv('TOMORROW_API_KEY')   

TICKETMASTER_Events_API_Endpoint = "https://app.ticketmaster.com/discovery/v2/events.json"
TICKETMASTER_Venues_API_Endpoint = "https://app.ticketmaster.com/discovery/v2/venues/.json"
TOMORROW_API_Endpoint = "https://api.tomorrow.io/v4/weather/forecast"


In [26]:
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

conn = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE
)

In [54]:
# In order to pull the next 7 days of events
# Get today's date
today = datetime.today()

# Get the date 7 days from today
next_7_days = today + timedelta(days=6)

# Format the dates in the correct format (e.g., 'YYYY-MM-DD')
start_date = today.strftime('%Y-%m-%dT00:00:00Z')
end_date = six_days_later.strftime('%Y-%m-%dT23:59:59Z')

print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2025-02-05T00:00:00Z
End Date: 2025-02-11T23:59:59Z


In [55]:
# EVENTS API
# API pull with parameters
params = {
    "apikey": TICKETMASTER_API_KEY,       
    "city": "San Diego", 
    "startDateTime": start_date,  # Start date (today)
    "endDateTime": end_date      # End date (7 days later)                 
}

# Send the request to the API
response = requests.get(TICKETMASTER_Events_API_Endpoint, params=params)

In [56]:
# Check if the request was successful
if response.status_code == 200:
    print("API call succeeded!")
    data = response.json()  # The API response in JSON format
    print(data)             # Print the data (you'll see the events info here)
else:
    print(f"API call failed with status code: {response.status_code}")
    print("Response details:", response.text)  # Print more details on the failure

API call succeeded!
{'_embedded': {'events': [{'name': "ArtPower Presents Ira Glass: Seven Things I've Learned", 'type': 'event', 'id': 'vvG1IZbCNi7Sj6', 'test': False, 'url': 'https://www.ticketmaster.com/artpower-presents-ira-glass-seven-things-san-diego-california-02-08-2025/event/0A0060E8EC133707', 'locale': 'en-us', 'images': [{'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/5eb/04356440-7939-4cde-8013-637f9796c5eb_1845291_EVENT_DETAIL_PAGE_16_9.jpg', 'width': 205, 'height': 115, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/5eb/04356440-7939-4cde-8013-637f9796c5eb_1845291_TABLET_LANDSCAPE_LARGE_16_9.jpg', 'width': 2048, 'height': 1152, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/5eb/04356440-7939-4cde-8013-637f9796c5eb_1845291_RETINA_LANDSCAPE_16_9.jpg', 'width': 1136, 'height': 639, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/5eb/04356440-7939-4cde-8013-637f9796c5eb_1845291_RETINA_PO

In [63]:
# Extract events data from JSON response
events = data["_embedded"]["events"]

# Create a DataFrame
df = pd.DataFrame(events)

# Flatten nested structures (e.g., "dates" and "sales")
df['start_date'] = df['dates'].apply(lambda x: x['start']['localDate'])
df['venue_name'] = df['_embedded'].apply(lambda x: x['venue'][0]['name'] if 'venue' in x else None)
df['venue_id'] = df['_embedded'].apply(lambda x: x['venue'][0]['id'] if 'venue' in x else None)
df['postal_code'] = df['_embedded'].apply(lambda x: x['venue'][0]['postalCode'] if 'venue' in x and 'postalCode' in x['venue'][0] else None)

# Handle missing or empty classifications
df['classification_name'] = df['classifications'].apply(lambda x: x[0]['segment']['name'] if isinstance(x, list) and x else None)

# Select relevant columns
df_table = df[['name', 'start_date', 'venue_name', 'venue_id', 'classification_name', 'postal_code']]

# Print the table
print(df_table)

                                                 name  start_date venue_name  \
0   ArtPower Presents Ira Glass: Seven Things I've...  2025-02-08       None   
1   La Jolla Music Society presents KODO: One Eart...  2025-02-06       None   
2                                           Alt Bloom  2025-02-07       None   
3                                           lou deezi  2025-02-09       None   
4             Marquise Knox with Lafayette Blues Band  2025-02-08       None   
5   UC San Diego Tritons Mens Basketball vs. Calif...  2025-02-06       None   
6                                        Maro - (21+)  2025-02-06       None   
7                                     Lo Moon - (21+)  2025-02-05       None   
8                                      VenueNext Test  2025-02-04       None   
9                                     2025 Group Tour  2025-02-06       None   
10                              2425 PC Fees - WICKED  2025-02-06       None   
11                                    20

In [67]:
pd.set_option('display.max_columns', None)
df_test = pd.DataFrame(events)
print(df_test.columns)
print(df_test)

Index(['name', 'type', 'id', 'test', 'url', 'locale', 'images', 'sales',
       'dates', 'classifications', 'promoter', 'promoters', 'info',
       'pleaseNote', 'priceRanges', 'products', 'seatmap', 'accessibility',
       'ticketLimit', 'ageRestrictions', 'ticketing', '_links', '_embedded',
       'outlets'],
      dtype='object')
                                                 name   type  \
0   ArtPower Presents Ira Glass: Seven Things I've...  event   
1   La Jolla Music Society presents KODO: One Eart...  event   
2                                           Alt Bloom  event   
3                                           lou deezi  event   
4             Marquise Knox with Lafayette Blues Band  event   
5   UC San Diego Tritons Mens Basketball vs. Calif...  event   
6                                        Maro - (21+)  event   
7                                     Lo Moon - (21+)  event   
8                                      VenueNext Test  event   
9                        